<h1> Segmenting and Clustering Neighborhoods in Toronto City </h1>

<h4>Importing dependencies</h4>

In [143]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<h4>Requesting source page</h4>

In [144]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

<h4>Using BeautifulSoup for scraping the data and geocoder for obtaining coordinates</h4>

In [152]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
postcode = []
borough = []
neighborhood = []
latitudes = []
longitudes = []
for row in table.findAll('tr'):
    temp = []
    for each_data in row.findAll('td'):
        temp.append(each_data.text)
    if len(temp) and temp[1] != 'Not assigned': #Ignoring cells with a borough that is Not assigned
        postcode.append(temp[0])
        borough.append(temp[1])
        temp_2 = temp[2].rstrip('\n')
        if temp_2 == 'Not assigned': #If a cell has a Not assigned neighborhood
            temp_2 = temp[1]          #then the neighborhood will be the same as the borough
        neighborhood.append(temp_2)
        
        #lat_lng_coords = None
        #while(lat_lng_coords is None):
            #g = geocoder.google('{}, Toronto, Ontario'.format(temp[0]))
            #lat_lng_coords = g.latlng

        #latitudes.append(lat_lng_coords[0])
        #longitudes.append(lat_lng_coords[1])
data = {'Postal Code': postcode, 'Borough': borough, 'Neighborhood': neighborhood} #The dataframe will consist of three columns:
                                                                                  #PostalCode, Borough, and Neighborhood

Given that this package can be very unreliable, in case you wish to get the geographical coordinates of the neighborhoods using the Geocoder package, uncomment the lines in the above code snippet

<h4>Storing the scraped data in pandas dataframe</h4>

In [153]:
df = pd.DataFrame(data)                                                           
df = df.groupby("Postal Code").agg(lambda x:','.join(set(x)))

<h4>The number of rows of the dataframe</h4>

In [155]:
df.shape

(103, 2)

<h4>Displaying any 10 data from the dataframe</h4>

In [156]:
df.sample(10)

,Borough,Neighborhood
Postal Code,,
M3B,North York,Don Mills North
M6P,West Toronto,"High Park,The Junction South"
M4B,East York,"Woodbine Gardens,Parkview Hill"
M1N,Scarborough,"Birch Cliff,Cliffside West"
M1J,Scarborough,Scarborough Village
M1T,Scarborough,"Tam O'Shanter,Sullivan,Clarks Corners"
M3L,North York,Downsview West
M5R,Central Toronto,"The Annex,Yorkville,North Midtown"
M3H,North York,"Wilson Heights,Downsview North,Bathurst Manor"


<h4>Get the latitude and the longitude coordinates of each neighborhood using geospatial data</h4>   
The http://cocl.us/Geospatial_data (csv file) contains geographical coordinates of each postal code of Toronto

In [157]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df = pd.merge(df, df2, on = 'Postal Code')
df.drop('Postal Code', 1,inplace=True)

<h4>Examing the resulting dataframe</h4> 

In [158]:
df.sample(10)

,Borough,Neighborhood,Latitude,Longitude
26,North York,Don Mills North,43.745906,-79.352188
18,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556
28,North York,"Wilson Heights,Downsview North,Bathurst Manor",43.754328,-79.442259
20,North York,"Silver Hills,York Mills",43.757490,-79.374714
58,Downtown Toronto,"Adelaide,Richmond,King",43.650571,-79.384568
25,North York,Parkwoods,43.753259,-79.329656
87,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
14,Scarborough,"Steeles East,Agincourt North,L'Amoreaux East,M...",43.815252,-79.284577
80,York,"Mount Dennis,Silverthorn,Del Ray,Keelesdale",43.691116,-79.476013
101,Etobicoke,"Albion Gardens,Mount Olive,Silverstone,Jamesto...",43.739416,-79.588437


In [159]:
df.groupby('Borough')['Neighborhood'].count()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [160]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [161]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<h4>Creating a map of Toronto with neighborhoods superimposed on top</h4>

In [162]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h4>Define Foursquare Credentials and Version</h4>

In [163]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = Creds['CLIENT_ID'][0] # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

In [164]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [165]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Port Union,Highland Creek,Rouge Hill
West Hill,Morningside,Guildwood
Woburn
Cedarbrae
Scarborough Village
Ionview,Kennedy Park,East Birchmount Park
Golden Mile,Oakridge,Clairlea
Cliffside,Scarborough Village West,Cliffcrest
Birch Cliff,Cliffside West
Wexford Heights,Dorset Park,Scarborough Town Centre
Maryvale,Wexford
Agincourt
Tam O'Shanter,Sullivan,Clarks Corners
Steeles East,Agincourt North,L'Amoreaux East,Milliken
L'Amoreaux West
Upper Rouge
Hillcrest Village
Oriole,Henry Farm,Fairview
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Wilson Heights,Downsview North,Bathurst Manor
York University,Northwood Park
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale,The Danforth West
India Bazaar,The Beac

In [166]:
print(toronto_venues.shape)
toronto_venues.head()

(2250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"West Hill,Morningside,Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"West Hill,Morningside,Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [167]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Albion Gardens,Mount Olive,Silverstone,Jamestown,South Steeles,Beaumond Heights,Humbergate,Thistletown",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
"Birch Cliff,Cliffside West",4,4,4,4,4,4
"Brockton,Parkdale Village,Exhibition Place",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18


In [168]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 281 uniques categories.


In [169]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,Richmond,King",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
2,"Albion Gardens,Mount Olive,Silverstone,Jamesto...",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
4,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
6,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
7,"Brockton,Parkdale Village,Exhibition Place",0.043478,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
8,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
9,"CFB Toronto,Downsview East",0.000000,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000


In [171]:
toronto_grouped.shape

(100, 281)

In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [179]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(100, 11)

In [180]:
# set number of clusters
kclusters = 11

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  9,  9,  1,  1,  1,  1,  1,  1,
        8,  1,  1,  1,  1,  1,  1,  1,  7,  1,  9,  1,  1,  1,  1,  1,  1,
        1,  1,  5, 10,  9,  1,  1,  1,  1,  1,  9,  1,  1,  1,  1,  0,  1,
        1,  9,  1,  1,  1,  1,  9,  1,  4,  1,  1,  1,  1,  9,  1,  6,  1,
        7,  2,  1,  1,  9,  9,  1,  1,  1,  1,  1,  1,  1, 10,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  9,  1])

In [181]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,6.0,Fast Food Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
1,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,1.0,Bar,History Museum,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
2,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711,1.0,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Electronics Store,Spa,Donut Shop,Doner Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Insurance Office,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Caribbean Restaurant,Thai Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant


In [188]:
toronto_merged.drop(toronto_merged[np.isnan(toronto_merged['Cluster Labels'])].index, inplace=True)

In [189]:
# create map
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [200]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,"Montgomery Road,Old Mill North,The Kingsway",River,Park,College Gym,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


In [201]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Port Union,Highland Creek,Rouge Hill",Bar,History Museum,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
2,"West Hill,Morningside,Guildwood",Rental Car Location,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Electronics Store,Spa,Donut Shop,Doner Restaurant
3,Woburn,Coffee Shop,Korean Restaurant,Insurance Office,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,Cedarbrae,Fried Chicken Joint,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Caribbean Restaurant,Thai Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant
6,"Ionview,Kennedy Park,East Birchmount Park",Department Store,Coffee Shop,Playground,Discount Store,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar
7,"Golden Mile,Oakridge,Clairlea",Bus Line,Bakery,Park,Metro Station,Fast Food Restaurant,Intersection,Bus Station,Soccer Field,Convenience Store,Cosmetics Shop
9,"Birch Cliff,Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
10,"Wexford Heights,Dorset Park,Scarborough Town C...",Indian Restaurant,Latin American Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
11,"Maryvale,Wexford",Middle Eastern Restaurant,Auto Garage,Bakery,Breakfast Spot,Sandwich Place,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
12,Agincourt,Lounge,Furniture / Home Store,Skating Rink,Breakfast Spot,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills,York Mills",Cafeteria,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant


In [197]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Willowdale,Newtonbrook",Piano Bar,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant


In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Princess Gardens,Martin Grove,Cloverdale,Islin...",Bank,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Department Store


In [211]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Humber Summit,Empanada Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


In [212]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Fast Food Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


In [213]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Grocery Store,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
33,Downsview Northwest,Gym / Fitness Center,Athletics & Sports,Liquor Store,Discount Store,Grocery Store,Airport Food Court,Dessert Shop,Accessories Store,Ethiopian Restaurant,Empanada Restaurant


In [214]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffside,Scarborough Village West,Cliffcrest",Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run


In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Steeles East,Agincourt North,L'Amoreaux East,M...",Park,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
23,York Mills West,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
25,Parkwoods,Park,Fast Food Restaurant,Food & Drink Shop,Dive Bar,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
30,"CFB Toronto,Downsview East",Park,Airport,Playground,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
40,East Toronto,Park,Pizza Place,Convenience Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
44,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Ethiopian Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar
50,Rosedale,Park,Playground,Building,Trail,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant,Dog Run,Curling Ice
73,Humewood-Cedarvale,Trail,Hockey Arena,Park,Field,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
74,Caledonia-Fairbanks,Park,Market,Fast Food Restaurant,Women's Store,Pharmacy,College Rec Center,College Stadium,Eastern European Restaurant,Dumpling Restaurant,Drugstore
79,"North Park,Upwood Park,Downsview",Park,Bakery,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 10, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"The Queensway East,Old Mill South,Mimico NE,Hu...",Baseball Field,Pool,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
97,"Humberlea,Emery",Baseball Field,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Department Store
